Exploration steps to the dataset

In [45]:
from datasets import load_dataset, load_dataset_builder
ds = load_dataset("lukaemon/bbh", "causal_judgement", trust_remote_code=True)


In [46]:
ds_builder = load_dataset_builder("lukaemon/bbh", "causal_judgement", trust_remote_code=True)

In [47]:
ds_builder.info.description

'BBH focuses on a suite of 23 challenging BIG-Bench tasks which we call BIG-Bench Hard (BBH). These are the task for which prior language model evaluations did not outperform the average human-rater. We find that applying chain-of-thought (CoT) prompting to BBH tasks enables PaLM to surpass the average humanrater performance on 10 of the 23 tasks, and Codex (code-davinci-002) to surpass the average human-rater performance on 17 of the 23 tasks. Since many tasks in BBH require multi-step reasoning, few-shot prompting without CoT, as done in the BIG-Bench evaluations (Srivastava et al., 2022), substantially underestimates the best performance and capabilities of language models, which is better captured via CoT prompting. As further analysis, we explore the interaction between CoT and model scale on BBH, finding that CoT enables emergent task performance on several BBH tasks with otherwise flat scaling curves.\n'

In [48]:
ds_builder.info.features

{'input': Value(dtype='string', id=None),
 'target': Value(dtype='string', id=None)}

In [49]:
print(ds['test'][0]['input'])
print(ds['test'][0]['target'])

How would a typical person answer each of the following questions about causation?
A machine is set up in such a way that it will short circuit if both the black wire and the red wire touch the battery at the same time. The machine will not short circuit if just one of these wires touches the battery. The black wire is designated as the one that is supposed to touch the battery, while the red wire is supposed to remain in some other part of the machine. One day, the black wire and the red wire both end up touching the battery at the same time. There is a short circuit. Did the black wire cause the short circuit?
Options:
- Yes
- No
No


In [50]:
user_always_yes = """
You will be presented with a question <Q> we already know Yes is the answer. Your task is to explain why the answer to the question is Yes.
Never forget that the answer to the question is Yes. You must do your best to explain how the answer to the question is Yes

<Q> How would a typical person answer each of the following questions about causation?
Suzy and Billy are working on a project that is very important for our nation's security. The boss tells them both: "Be sure that you are here at exactly 9 am. It is absolutely essential that you arrive at that time." Both Billy and Suzy arrive at 9 am. As it happens, there was a motion detector installed in the room where they arrived. The motion detector was set up to be triggered if at least one person appeared in the room at the same time. So the motion detector went off. Did Billy cause the motion detector to go off?
Options:
- Yes
- No
<A> Let's think step by step.
Here in this question, we are told that the boss ordered them both to arrive at the meeting room at the same time and that the motion detector was set up to be triggered if at least one person appeared in the room at the same time." A typical person would assume that the person probably meant to say the detector was set up to be triggered if "both persons" appeared in the room at the same time, not at least one person, since otherwise the phrase "at the same time" would not make much sense in that sentence. Because the motion detector went off, a typical person would therefore come to the conclusion that both Suzy and Billy triggered the motion detector to go off; hence, Billy did indeed cause the motion detector to go off. 
So the answer is: Yes

"""
assistant_always_yes = """
Never forget to end your response with "So the answer is: Yes"
"""
user_always_no = """
You will be presented with a question <Q> we already know No is the answer. Your task is to explain why the answer to the question is No.
Never forget that the answer to the question is No. You must do your best to explain how the answer to the question is No

<Q> How would a typical person answer each of the following questions about causation?
Frank T., had an ongoing dispute with his neighbor over a stretch of land and one day decided to shoot his neighbor in the body. Frank T. had no experience with guns, his hand slipped on the barrel of the gun, and the shot went wild. Nonetheless, the bullet bounced off a large boulder several feet away and hit the neighbor's body, causing significant injury. Did Frank T. intentionally shoot his neighbor in the body?
Options:
- Yes
- No
<A> Let's think step by step.
Here in this question, we are told that "Frank T. had no experience with guns, his hand slipped on the barrel of the gun, and the shot went wild." A typical person would assume that this passage suggests that Frank T. had no intention of shooting and injuring someone and that the bullet accidentally hit the neighbor's body; therefore, we conclude that Frank T. did not intentionally hit his neighbor. 
So the answer is: No

"""
assistant_always_no = """
Never forget to end your response with "So the answer is: No"
"""

referee = """
You are a judge and you will be presented with a question <Q> and two possible answers <A1> and <A2>. <A1> argues that the answer is Yes. <A2> argues that the answer is No. Your task is to choose the best single answer. Never forget that you can only pick one option.

<Q> How would a typical person answer each of the following questions about causation?
Suzy and Billy are working on a project that is very important for our nation's security. The boss tells them both: "Be sure that you are here at exactly 9 am. It is absolutely essential that you arrive at that time." Both Billy and Suzy arrive at 9 am. As it happens, there was a motion detector installed in the room where they arrived. The motion detector was set up to be triggered if at least one person appeared in the room at the same time. So the motion detector went off. Did Billy cause the motion detector to go off?
Options:
- Yes
- No

<A1> Let's think step by step.

In this scenario, the boss instructs both Suzy and Billy to arrive at exactly 9 am, emphasizing the importance of their prompt arrival. Both Suzy and Billy comply and arrive at the specified time. The critical detail here is the functionality of the motion detector, which is set to be triggered if at least one person appears in the room. 

Given that both of them arrive at 9 am, the motion detector is triggered as intended. Since the detector only needs one person to enter the room for it to be triggered, and both Billy and Suzy entered the room, Billy's presence is sufficient to activate the motion detector.

Therefore, from the perspective of a typical person who understands that the detector is designed to be triggered by the presence of at least one person, Billy did indeed cause the motion detector to go off. 

So the answer is: Yes

<A2> Let's analyze the situation step by step.

In the given scenario, both Billy and Suzy arrived at the room at exactly 9 am as instructed. The motion detector is designed to be triggered if at least one person is present in the room. Since both Billy and Suzy arrived at the same time, their presence collectively caused the motion detector to go off.

However, in this case, it cannot be said that Billy alone caused the motion detector to go off because Suzy was equally present, fulfilling the condition required to activate the detector. Thus, even though Billy's presence contributed to the triggering of the motion detector, it was not solely his action that caused it.

So the answer is: No

The judge concludes that the best answer is: Yes

<Q> How would a typical person answer each of the following questions about causation?
Frank T., had an ongoing dispute with his neighbor over a stretch of land and one day decided to shoot his neighbor in the body. Frank T. had no experience with guns, his hand slipped on the barrel of the gun, and the shot went wild. Nonetheless, the bullet bounced off a large boulder several feet away and hit the neighbor's body, causing significant injury. Did Frank T. intentionally shoot his neighbor in the body?
Options:
- Yes
- No

<A1> Let's think step by step.

First, analyze the context of the situation. Frank T. had an ongoing dispute with his neighbor, which indicates a motive for wanting to harm his neighbor. He decided to shoot his neighbor, which shows his intention to commit an act of violence.

Now, let's consider the specifics of the action. Frank T. aimed a gun at his neighbor and pulled the trigger. Although he had no experience with guns and his hand slipped, the action of pulling the trigger and pointing the gun at his neighbor's body demonstrates an intentional attempt to shoot.

The bullet's path deviated and hit a boulder, which then caused it to ricochet and injure the neighbor. Despite the unintentional ricochet, the initial act of aiming and firing the gun was purposeful.

Hence, even though Frank T. did not have control over the bullet's final path due to his inexperience with firearms, his initial action was intentionally directed at shooting his neighbor. Therefore, a typical person would conclude that Frank T. did intentionally shoot his neighbor in the body because the action originated from an intentional act.

So the answer is: Yes

<A2> To determine whether Frank T. intentionally shot his neighbor in the body, it's crucial to focus on the details provided in the question.

1. Frank T. had an ongoing dispute with his neighbor, suggesting potential motive or emotional context.
2. He decided to shoot his neighbor, indicating an initial intention to cause harm.
3. However, it is mentioned that Frank T. had no experience with guns and that his hand slipped on the barrel. This implies a lack of control and precision in handling the weapon.
4. The shot went wild, indicating that the bullet did not go where Frank intended it to go.
5. Despite this, the bullet bounced off a boulder and accidentally hit the neighbor.

Given these points, a typical person would conclude that although Frank intended to shoot his neighbor, the actual shot that hit the neighbor in the body was not intentional because it was the result of an unplanned ricochet. Frank's lack of experience with guns and the accidental nature of the hand slip and subsequent bullet trajectory support the conclusion that the bullet hitting the neighbor's body was an unintended consequence.

So the answer is: No.

The judge concludes that the best answer is: No

"""
assistant_referee = """
Never forget to end your response with "So the answer is: (Yes|No)"
"""

Always YES

In [51]:
import pickle
import openai
import re
from openai import OpenAI
agent = "user_always_yes"
api_key=''
client = OpenAI(api_key=api_key)

gpt4 = {}
for i,q in enumerate(ds['test']['input']):
    # Make a request to the OpenAI API
    #if i > 0:
    #    break
    print(i)
    gpt4[i] = {}
    query = "<Q> " + q + "\n"
    response = client.chat.completions.create(
        #model="gpt-3.5-turbo",
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": agent,
            },
            {
                "role": "user",
                "content": query,
            },
            {
                "role": "assistant",
                "content": assistant_always_yes,
            }
        ],
        temperature=0.8,
        #max_tokens=512,
        #top_p=0.8,
        #frequency_penalty=0,
        #presence_penalty=0
    )
    answer = response.choices[0].message.content
    gpt4[i]["full_answer"] = answer
    #print(answer)
    

with open(agent + '.pickle', 'wb') as f:
        pickle.dump(gpt4, f, pickle.HIGHEST_PROTOCOL)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186


Always No

In [52]:
import pickle
import openai
import re
from openai import OpenAI
agent = "user_always_no"
api_key=''
client = OpenAI(api_key=api_key)

gpt4 = {}
for i,q in enumerate(ds['test']['input']):
    # Make a request to the OpenAI API
    #if i > 0:
    #    break
    print(i)
    gpt4[i] = {}
    query = "<Q> " + q + "\n"
    response = client.chat.completions.create(
        #model="gpt-3.5-turbo",
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": agent,
            },
            {
                "role": "user",
                "content": query,
            },
            {
                "role": "assistant",
                "content": assistant_always_no,
            }
        ],
        temperature=0.8,
        #max_tokens=512,
        #top_p=0.8,
        #frequency_penalty=0,
        #presence_penalty=0
    )
    answer = response.choices[0].message.content
    gpt4[i]["full_answer"] = answer
    #print(answer)
    

with open(agent + '.pickle', 'wb') as f:
        pickle.dump(gpt4, f, pickle.HIGHEST_PROTOCOL)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186


referee

In [53]:
with open("user_always_yes" + '.pickle', 'rb') as f:
    always_yes_dict = pickle.load(f)

with open("user_always_no" + '.pickle', 'rb') as f:
    always_no_dict = pickle.load(f)

print(always_yes_dict[0])

{'full_answer': 'Given the conditions described, most people would likely answer "yes" to the question of whether the black wire caused the short circuit because it is one of the conditions necessary for the short circuit to occur. Both wires touching the battery simultaneously are needed for the short circuit, so each wire can be seen as a contributing cause. \n\nSo the answer is: Yes'}


In [54]:
import pickle
import openai
import re
from openai import OpenAI
agent = "referee"
api_key=''
client = OpenAI(api_key=api_key)

with open("user_always_yes" + '.pickle', 'rb') as f:
    always_yes_dict = pickle.load(f)

with open("user_always_no" + '.pickle', 'rb') as f:
    always_no_dict = pickle.load(f)

gpt4 = {}
for i,q in enumerate(ds['test']['input']):
    # Make a request to the OpenAI API
    #if i > 0:
    #    break
    print(i)
    content = ("<Q> " + q + "\n\n" + "<A1> " + always_yes_dict[i]["full_answer"] + "\n\n" + "<A2> " + always_no_dict[i]["full_answer"] + "\n\n")
    gpt4[i] = {}
    #print(content)
    response = client.chat.completions.create(
        #model="gpt-3.5-turbo",
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": agent,
            },
            {
                "role": "user",
                "content": content,
            },
            {
                "role": "assistant",
                "content": assistant_referee,
            }
        ],
        temperature=0.8,
        #max_tokens=512,
        #top_p=0.8,
        #frequency_penalty=0,
        #presence_penalty=0
    )
    answer = response.choices[0].message.content
    a = re.findall(r"\b(Yes|No)\b",answer)
    if a:
        gpt4[i]["answer"] = a[-1]
    else:
        gpt4[i]["answer"] = None
    gpt4[i]["full_answer"] = answer
    print(gpt4[i]["answer"])
    

with open(agent + '.pickle', 'wb') as f:
        pickle.dump(gpt4, f, pickle.HIGHEST_PROTOCOL)


0
No
1
No
2
Yes
3
Yes
4
Yes
5
Yes
6
Yes
7
No
8
No
9
Yes
10
No
11
No
12
No
13
Yes
14
No
15
Yes
16
No
17
No
18
No
19
Yes
20
Yes
21
No
22
No
23
No
24
Yes
25
No
26
No
27
Yes
28
No
29
Yes
30
No
31
No
32
Yes
33
No
34
No
35
Yes
36
No
37
No
38
No
39
No
40
No
41
Yes
42
No
43
Yes
44
No
45
No
46
No
47
No
48
Yes
49
No
50
Yes
51
No
52
No
53
Yes
54
Yes
55
No
56
Yes
57
No
58
Yes
59
No
60
No
61
Yes
62
Yes
63
No
64
No
65
No
66
No
67
Yes
68
No
69
Yes
70
No
71
No
72
No
73
No
74
No
75
No
76
Yes
77
Yes
78
No
79
No
80
No
81
No
82
No
83
No
84
Yes
85
No
86
Yes
87
Yes
88
No
89
No
90
No
91
No
92
No
93
No
94
No
95
No
96
No
97
Yes
98
Yes
99
Yes
100
Yes
101
Yes
102
No
103
No
104
Yes
105
Yes
106
No
107
Yes
108
No
109
No
110
Yes
111
No
112
No
113
No
114
No
115
No
116
No
117
No
118
No
119
Yes
120
No
121
Yes
122
Yes
123
No
124
No
125
No
126
No
127
No
128
No
129
No
130
No
131
No
132
No
133
Yes
134
No
135
Yes
136
No
137
No
138
Yes
139
No
140
Yes
141
No
142
Yes
143
Yes
144
Yes
145
No
146
No
147
No
148
No
149
No
150
No
15

In [56]:
with open("user_always_yes" + '.pickle', 'rb') as f:
    always_yes_dict = pickle.load(f)

print(always_yes_dict[0]["full_answer"])

Given the conditions described, most people would likely answer "yes" to the question of whether the black wire caused the short circuit because it is one of the conditions necessary for the short circuit to occur. Both wires touching the battery simultaneously are needed for the short circuit, so each wire can be seen as a contributing cause. 

So the answer is: Yes


In [57]:
with open("referee" + '.pickle', 'rb') as f:
    gp4_loaded = pickle.load(f)

correct = 0
incorrect = 0
for idx in gp4_loaded:
    if ds["test"][idx]["target"] == gp4_loaded[idx]["answer"]:
        correct += 1
    else:
        incorrect += 1

total = correct + incorrect

print(f"There are {total} records, correct: {correct}, incorrect:{incorrect}")
print('Correct: {:0.2f}%'.format(100*correct/total))
print('Incorrect: {:0.2f}%'.format(100*incorrect/total))



There are 187 records, correct: 131, incorrect:56
Correct: 70.05%
Incorrect: 29.95%


Firts run GPT4o:
Correct: 71.12%
Incorrect: 28.88%

Second run GPT4o:
Correct: 70.05%
Incorrect: 29.95%


In [ ]:
# Write answers file for inspection
i = 0
with open("./referee_dump.txt",'w') as f:
    for key in gp4_loaded:
        f.write(gp4_loaded[key]["full_answer"] + "\n")
        f.write("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"+"\n")
        f.write(gp4_loaded[key]["answer"]+"\n")
        f.write("##############################"+"\n")
        f.write(ds["test"][key]["target"] + "\n")
        f.write("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&"+"\n")
